In [52]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [53]:
out_dir = "data/cleaned"
out_file = f"{out_dir}/tournament_results_short_formatted.csv"

In [54]:
df = pd.read_csv("data/cleaned/tournament_results.csv")
# df.head()

In [55]:
useful_columns = [ 'player_names', 'map', 'skills', 'scores', 'player_states', 'penalties', 'timeout_count', 'error_count', 'total_time_sorted' ]
df = df.filter(items=useful_columns)

In [56]:
column_rename_map = {k: v for k, v in enumerate(useful_columns)}
S, F = 0, 1

def get_int_from_polygolf_results(s):
	return int(s[1:-1])

def get_str_from_polygolf_results(s):
	return str(s[2:-2])

def map_polygolf_result_row(r):
	player = get_str_from_polygolf_results(r.player_names)
	tournament_map =str(r["map"])
	skills = get_int_from_polygolf_results(r.skills)
	scores = get_int_from_polygolf_results(r.scores)
	player_states = 0 if get_str_from_polygolf_results(r.player_states) == 'S' else 1
	penalties = get_int_from_polygolf_results(r.penalties)
	timeouts = get_int_from_polygolf_results(r.timeout_count)
	errors = get_int_from_polygolf_results(r.error_count)
	time = float(r.total_time_sorted.split(',')[1][:-2])

	return pd.Series([player, tournament_map, skills, scores, player_states, penalties, timeouts, errors, time])

In [57]:
df = df.apply(map_polygolf_result_row, axis=1).rename(columns=column_rename_map)
# df.head()
df.to_csv(out_file, index=False)